In [3]:
import pandas as pd
import json 
import matplotlib.pyplot as plt
import matplotlib
import re
import string
from nltk.corpus import stopwords
import regex
import nltk
from datetime import timedelta
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [13]:
# import raw that of tweets from twitter API for for each 
df0 = pd.read_csv('2_raw_twitter_api_data/0503_tweets.csv')
df0.head(3)

,username,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,favoritecount,text,hashtags,retweetuser
0,SilvestriF,"Córdoba, Argentina",3083,1731,47901,2011-03-05 17:57:01,2021-05-03 23:59:58,375,0,"True or false, Bitcoin will be over $100,000 b...",[],BitcoinBF
1,horak18,NaN,662,331,7593,2014-10-17 17:58:30,2021-05-03 23:59:57,54,0,The bitcoin mining community needs to organize...,[],kevinolearytv
2,CryptoWatchBot,"New York, NY",2080,2106,4222,2018-01-22 00:24:47,2021-05-03 23:59:57,1,5,#Investing 65.9% in this optimal #crypto portf...,"[{'text': 'Investing', 'indices': [0, 10]}, {'...",NaN


In [8]:
#create columns for date, hour, and minute
df0['date'] = df0.tweetcreatedts.apply(lambda x:  str(x)[:10])
df0['date'] = pd.to_datetime(df0.date)
df0['hour'] = df0.tweetcreatedts.apply(lambda x:  str(x)[10:13])
df0['minute'] = df0.tweetcreatedts.apply(lambda x:  str(x)[14:16])
df0.head(3)

,username,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,favoritecount,text,hashtags,retweetuser,date,hour,minute
0,SilvestriF,"Córdoba, Argentina",3083,1731,47901,2011-03-05 17:57:01,2021-05-03 23:59:58,375,0,"True or false, Bitcoin will be over $100,000 b...",[],BitcoinBF,2021-05-03,23,59
1,horak18,NaN,662,331,7593,2014-10-17 17:58:30,2021-05-03 23:59:57,54,0,The bitcoin mining community needs to organize...,[],kevinolearytv,2021-05-03,23,59
2,CryptoWatchBot,"New York, NY",2080,2106,4222,2018-01-22 00:24:47,2021-05-03 23:59:57,1,5,#Investing 65.9% in this optimal #crypto portf...,"[{'text': 'Investing', 'indices': [0, 10]}, {'...",NaN,2021-05-03,23,59


In [9]:
#cleaning process
#### Remove Links and User Tags
find_links = re.compile(r'http\S+')
find_users = re.compile(r'@\w+')
find_hashtags = re.compile(r'#\w+')
df0['text_nolink'] = [find_links.sub("",x) for x in df0['text']]
df0['text_nolink'] = [find_users.sub("",x) for x in df0['text_nolink']]
df0['text_nolink'] = [find_hashtags.sub("",x) for x in df0['text_nolink']]

In [10]:
#### Remove punctuation and lowercase
string.punctuation
remv_punc = str.maketrans('','',string.punctuation + '“' +"‘"+'”')
df0['text_lower'] =  df0['text_nolink'].str.lower().str.replace("\n","")
df0['text_clean'] = df0.text_lower.str.translate(remv_punc)

In [11]:
sid = SentimentIntensityAnalyzer()
pol_scores = [sid.polarity_scores(x)['compound'] for x in df0['text_clean']]
df0['sentiment'] = pd.Series(pol_scores)
df0

,username,location,following,followers,totaltweets,usercreatedts,tweetcreatedts,retweetcount,favoritecount,text,hashtags,retweetuser,date,hour,minute,text_nolink,text_lower,text_clean,sentiment
0,SilvestriF,"Córdoba, Argentina",3083,1731,47901,2011-03-05 17:57:01,2021-05-03 23:59:58,375,0,"True or false, Bitcoin will be over $100,000 b...",[],BitcoinBF,2021-05-03,23,59,"True or false, Bitcoin will be over $100,000 b...","true or false, bitcoin will be over $100,000 b...",true or false bitcoin will be over 100000 by t...,0.4215
1,horak18,NaN,662,331,7593,2014-10-17 17:58:30,2021-05-03 23:59:57,54,0,The bitcoin mining community needs to organize...,[],kevinolearytv,2021-05-03,23,59,The bitcoin mining community needs to organize...,the bitcoin mining community needs to organize...,the bitcoin mining community needs to organize...,0.4588
2,CryptoWatchBot,"New York, NY",2080,2106,4222,2018-01-22 00:24:47,2021-05-03 23:59:57,1,5,#Investing 65.9% in this optimal #crypto portf...,"[{'text': 'Investing', 'indices': [0, 10]}, {'...",NaN,2021-05-03,23,59,65.9% in this optimal portfolio and 34.1% i...,65.9% in this optimal portfolio and 34.1% i...,659 in this optimal portfolio and 341 in th...,0.4357
3,FreakieF,NaN,3744,1117,20553,2019-06-10 14:18:51,2021-05-03 23:59:57,73,0,"""ETH ATH!""... ETH'S ATH WAS 0.1564 BTC IN JUNE...",[],elpasobitcoin,2021-05-03,23,59,"""ETH ATH!""... ETH'S ATH WAS 0.1564 BTC IN JUNE...","""eth ath!""... eth's ath was 0.1564 btc in june...",eth ath eths ath was 01564 btc in june 2017 et...,0.4404
4,Therealchecker7,"England, United Kingdom",668,160,5463,2017-02-27 12:12:45,2021-05-03 23:59:57,1,1,@kgreifeld @rychardHzt #Ethereum is your chanc...,"[{'text': 'Ethereum', 'indices': [23, 32]}, {'...",NaN,2021-05-03,23,59,is your chance to buy the new at a price w...,is your chance to buy the new at a price w...,is your chance to buy the new at a price w...,0.6808
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,Amobnc,"Porto, Portugal",138,10,108,2020-08-06 23:28:19,2021-05-03 20:12:32,262,0,"Back above the 1 trillion $ threshold, #Bitcoi...","[{'text': 'Bitcoin', 'indices': [57, 65]}]",WClementeIII,2021-05-03,20,12,"Back above the 1 trillion $ threshold, is bui...","back above the 1 trillion $ threshold, is bui...",back above the 1 trillion threshold is build...,-0.2960
19996,AB09078,NaN,297,83,457,2011-07-07 10:11:51,2021-05-03 20:12:32,11,0,I think #IOTA will fly when it becomes widely ...,"[{'text': 'IOTA', 'indices': [23, 28]}]",outbreeak,2021-05-03,20,12,I think will fly when it becomes widely avail...,i think will fly when it becomes widely avail...,i think will fly when it becomes widely avail...,0.8847
19997,CRYPTOBOYMOON,"Islamabad, Pakistan",270,2640,162921,2016-11-10 08:47:36,2021-05-03 20:12:31,10,0,"#Ethereum's strong rally above $3,300 has shif...","[{'text': 'Ethereum', 'indices': [19, 28]}]",Cointelegraph,2021-05-03,20,12,"'s strong rally above $3,300 has shifted the f...","'s strong rally above $3,300 has shifted the f...",s strong rally above 3300 has shifted the focu...,0.2023
19998,ByteFederal,"Sarasota, FL",488,621,2605,2016-12-12 22:28:35,2021-05-03 20:12:30,121,0,You aren’t just saving money with Bitcoin. Bit...,[],ObiWanKenoBit,2021-05-03,20,12,You aren’t just saving money with Bitcoin. Bit...,you aren’t just saving money with bitcoin. bit...,you aren’t just saving money with bitcoin bitc...,0.0000


In [12]:
sent_ts = df0.set_index("tweetcreatedts").sort_index()
df1=df0.groupby(["date","hour",'minute'])['sentiment'].mean()
df1

date        hour  minute
2021-05-03   20   12        0.144689
                  13        0.233488
                  14        0.193711
                  15        0.208956
                  16        0.203408
                              ...   
             23   55        0.166587
                  56        0.174543
                  57        0.155319
                  58        0.190247
                  59        0.177563
Name: sentiment, Length: 228, dtype: float64

In [14]:
df1.to_csv('3_sentiment_score_output/0503score.csv') #do this for each days dataset (need to change the output name)